In [1]:
import pandas as pd 
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn import svm, metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn import preprocessing
from keras.utils import np_utils

import matplotlib.pyplot as plt

from sklearn.metrics import precision_recall_fscore_support
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import accuracy_score
from keras.models import model_from_json
from keras.models import load_model
from keras.constraints import max_norm
from sklearn.utils import class_weight

import seaborn as sns

Using TensorFlow backend.
/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:472: FutureWarn

In [2]:
sample= 125
Nfeatures= 513

In [3]:
def readCSV(inCSV1, inCSV0):
    print(inCSV1)
    df1= pd.read_csv(inCSV1)
    print(df1.shape)
    N_sample= df1.shape[0]
    print(N_sample)
    print(inCSV0)
    
    df= pd.read_csv(inCSV0)
    print(df.shape)
    
    #df= df.sample(N_sample)# under sample
    
    df= df.append(df1, ignore_index=True)
    print(df.shape)
    return df

In [4]:
inTrain1= '/Volumes/E/download/full/train1.csv'
inTrain0= '/Volumes/E/download/full/train0.csv'
dfTrain= readCSV(inTrain1, inTrain0)

/Volumes/E/download/full/train1.csv
(5780, 64126)
5780
/Volumes/E/download/full/train0.csv
(12254, 64126)
(18034, 64126)


In [5]:
inTest1= '/Volumes/E/download/full/test1.csv'
inTest0= '/Volumes/E/download/full/test0.csv'
dfTest= readCSV(inTest1, inTest0)

/Volumes/E/download/full/test1.csv
(1952, 64126)
1952
/Volumes/E/download/full/test0.csv
(4846, 64126)
(6798, 64126)


In [6]:
def readDF(df, NSamples):
    df= df.sample(NSamples)
    y= df["PHQ_Binary"].to_numpy()
    df_Nolabel= df.drop(['PHQ_Binary'], axis=1)
    X= df_Nolabel.to_numpy().reshape( NSamples, sample, Nfeatures, 1)
    print(X.shape)
    return X, y 

### start training

In [ ]:
NSamplesTrain= 18000
NSamplesTest= 6700
# NSamplesTrain= 200
# NSamplesTest= 100
X_train, y_train= readDF(dfTrain, NSamplesTrain)
X_test, y_test= readDF(dfTest, NSamplesTest) 

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras.regularizers import l2

In [ ]:
def cnn(X_train, y_train, X_test, y_test, batch_size, epochs, input_shape):
    nb_classes= 2
    L_weight= 0.01
    
    model = Sequential()
    model.add( Conv2D(32, (3, 3), padding='valid', strides=1, 
                     kernel_constraint=max_norm(3), bias_constraint=max_norm(3),
                     bias_regularizer=l2(L_weight),
                     input_shape=input_shape, activation='relu'))
    
    model.add(Dropout(0.3))
    model.add(MaxPooling2D(pool_size=(1, 3), strides=(1, 3)))
    
    model.add( Conv2D(32, (3, 3), padding='valid', strides=1, 
                     kernel_constraint=max_norm(3), bias_constraint=max_norm(3),
                     bias_regularizer=l2(L_weight),
                     input_shape=input_shape, activation='relu'))
    model.add(Dropout(0.3))
    model.add(MaxPooling2D(pool_size=(1, 3), strides=(1, 3)))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.6)) 

    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.6))

    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])
    
    class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)
    
    history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs,
                        class_weight=class_weights,
                        verbose=1, validation_data=(X_test, y_test))

    return model, history

In [ ]:
batch_size = 32
epochs = 3
print(batch_size, epochs)

input_shape= (sample, Nfeatures, 1)
input_shape

In [ ]:
model, history= cnn(X_train, y_train, X_test, y_test, batch_size, epochs, input_shape)

In [ ]:
def accuracy_curve(h):
    acc, loss, val_acc, val_loss = h.history['acc'], h.history['loss'], h.history['val_acc'], h.history['val_loss']
    epoch = len(acc)
    plt.figure(figsize=(17, 5))
    
    plt.subplot(122)
    plt.plot(range(epoch), loss, label='Train')
    plt.plot(range(epoch), val_loss, label='val')
    plt.title('Loss over ' + str(epoch) + ' Epochs', size=15)
    plt.legend()
    plt.grid(True)
    
    plt.subplot(121)
    plt.plot(range(epoch), acc, label='Train')
    plt.plot(range(epoch), val_acc, label='val')
    plt.title('Accuracy over ' + str(epoch) + ' Epochs', size=15)
    plt.legend()
    plt.grid(True)
    

    plt.show()
    
accuracy_curve(history)

In [ ]:
def plot_auc(y_test, y_score):
    
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    thresholds = dict()

    for i in range(2):
        fpr[i], tpr[i], thresholds[i] = roc_curve(y_test[:, i], y_score[:, i])
        #print(thresholds)
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

    plt.figure()
    lw = 2
    plt.plot(fpr[1], tpr[1], color='darkorange',
             lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[1])

    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()
    
    return

def modelMetrics(y_test, y_score):
    y_prob= [ y[1] for y in y_score]
    #print(y_prob)
    sns.distplot(y_prob);
    y_prob_mean= np.mean(y_prob)
    print("mean: ", y_prob_mean)
    
    y_pred= [ int(y > y_prob_mean) for y in y_prob]
    accuracy= accuracy_score(y_test, y_pred)
    precision, recall, fscore, _= precision_recall_fscore_support(y_test, y_pred, average='binary')

    print("accuracy: %.2f%%" % (accuracy*100))
    print("precision: %.2f%%" % (precision*100))
    print("recall: %.2f%%" % (recall*100))
    print("fscore: %.2f%%" % (fscore*100))
    print("Confusion matrix:\n%s" % metrics.confusion_matrix(y_test, y_pred))

def valuationMetrics(model, X_test, y_test):
    y_prob = model.predict(X_test, batch_size=32, verbose=1)
    y_test_ = np.eye(2)[y_test]
    plot_auc(y_test_, y_prob)
    modelMetrics(y_test, y_prob)     

In [ ]:
valuationMetrics(model, X_test, y_test)

In [ ]:
path= '../../media/model/depression.h5'
model.save(path)
print(path)

In [ ]:
model_load = load_model(path)